In [1]:
# 병렬 코퍼스 데이터에 대한 이해와 전처리

In [1]:
import pandas as pd

# lines= pd.read_csv('fra.txt', names=['src', 'tar', ''], sep='\t')  # 학습할 데이터 위치
# len(lines)

# # 전체 샘플의 개수
# lines = lines[0:5000] # 6만개만 저장
# lines.sample(10)

# # <sos>와 <eos> 대신 \t를 시작 심볼, \n을 종료 심볼로 간주하여 다시 출력
# lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
# lines.sample(10)

TRAIN_CSV_PATH = 'train.csv' # Train 데이터 위치
TEST_CSV_PATH = 'test.csv' # Test 데이터 위치

lines= pd.read_csv(TRAIN_CSV_PATH, names=['src', 'tar'], sep=',', skiprows=[0])  # 학습할 데이터 위치
len(lines)

# 전체 샘플의 개수
lines = lines[0:5000] # 6만개만 저장
lines.sample(4)

# <sos>와 <eos> 대신 \t를 시작 심볼, \n을 종료 심볼로 간주하여 다시 출력
lines.tar = lines.tar.apply(lambda x : '\t '+ x + ' \n')
lines.sample(4)

,src,tar
530,Help extends checkout.,\t (help) .> (checkout) : extends \n
29,The checkouts require payment.,\t (checkout) .> (payment) : include \n
1384,The customer does checkout.,\t customer -- (checkout) \n
1082,Help extends checkout.,\t (help) .> (checkout) : extends \n


In [2]:
# 글자 집합 구축
src_vocab=set()
for line in lines.src: # 1줄씩 읽음
    for char in line: # 1개의 글자씩 읽음
        src_vocab.add(char)

tar_vocab=set()
for line in lines.tar:
    for char in line:
        tar_vocab.add(char)

{'i', 'q', 'r', 'l', 'k', 'x', 'C', 'm', 'p', 'T', 'e', 'o', 'a', '.', 's', 'h', 'u', 't', 'H', 'c', 'b', 'd', ' ', 'y', 'n'}
{':', 'i', 'r', 'l', 'k', 'x', 'm', 'p', 'o', 'e', '\t', 'a', '.', 's', 'h', 'u', 't', 'c', '\n', '>', 'n', ')', 'd', '(', ' ', 'y', '-'}


In [4]:
# 글자 집합의 크기 확인
src_vocab_size = len(src_vocab)+1
tar_vocab_size = len(tar_vocab)+1
print(src_vocab_size)
print(tar_vocab_size)

26
28


In [5]:
# 각 글자에 인덱스 부여
src_to_index = dict([(word, i+1) for i, word in enumerate(src_vocab)])
tar_to_index = dict([(word, i+1) for i, word in enumerate(tar_vocab)])
print(src_to_index)
print(tar_to_index)

{'d': 1, 'l': 2, 'r': 3, 'C': 4, 'm': 5, 'e': 6, 'a': 7, 'p': 8, 'i': 9, 'x': 10, 'h': 11, 't': 12, 'n': 13, 'k': 14, ' ': 15, 'H': 16, 'o': 17, '.': 18, 's': 19, 'b': 20, 'y': 21, 'u': 22, 'q': 23, 'T': 24, 'c': 25}
{'d': 1, 'l': 2, 'r': 3, '-': 4, '\n': 5, 'm': 6, ':': 7, '>': 8, 'e': 9, 'a': 10, 'p': 11, 'i': 12, 'x': 13, 'h': 14, '\t': 15, '(': 16, ')': 17, 't': 18, 'n': 19, 'k': 20, ' ': 21, 'o': 22, '.': 23, 's': 24, 'y': 25, 'u': 26, 'c': 27}


In [6]:
# 인코더의 입력이 될 한글 문장 샘플에 대해서 정수 인코딩 수행
# 5개의 샘플만 출력
encoder_input = []
for line in lines.src: #입력 데이터에서 1줄씩 문장을 읽음
    temp_X = []
    for w in line: #각 줄에서 1개씩 글자를 읽음
        temp_X.append(src_to_index[w]) # 글자를 해당되는 정수로 변환
    encoder_input.append(temp_X)
print(encoder_input[:5])

[[24, 11, 6, 15, 25, 22, 19, 12, 17, 5, 6, 3, 15, 1, 17, 6, 19, 15, 25, 11, 6, 25, 14, 17, 22, 12, 18], [24, 11, 6, 15, 25, 11, 6, 25, 14, 17, 22, 12, 19, 15, 3, 6, 23, 22, 9, 3, 6, 15, 8, 7, 21, 5, 6, 13, 12, 18], [16, 6, 2, 8, 15, 6, 10, 12, 6, 13, 1, 19, 15, 25, 11, 6, 25, 14, 17, 22, 12, 18], [4, 11, 6, 25, 14, 17, 22, 12, 15, 9, 19, 15, 1, 17, 13, 6, 15, 20, 21, 15, 25, 2, 6, 3, 14, 18], [24, 11, 6, 15, 25, 22, 19, 12, 17, 5, 6, 3, 15, 1, 17, 6, 19, 15, 25, 11, 6, 25, 14, 17, 22, 12, 18]]


In [7]:
# 디코더의 입력이 될 영어 데이터에 대해서 정수 인코딩 수행
decoder_input = []
for line in lines.tar:
    temp_X = []
    for w in line:
        temp_X.append(tar_to_index[w])
    decoder_input.append(temp_X)
print(decoder_input[:5])

[[15, 21, 27, 26, 24, 18, 22, 6, 9, 3, 21, 4, 4, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 5], [15, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 23, 8, 21, 16, 11, 10, 25, 6, 9, 19, 18, 17, 21, 7, 21, 12, 19, 27, 2, 26, 1, 9, 21, 5], [15, 21, 16, 14, 9, 2, 11, 17, 21, 23, 8, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 7, 21, 9, 13, 18, 9, 19, 1, 24, 21, 5], [15, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 4, 4, 21, 27, 2, 9, 3, 20, 21, 5], [15, 21, 27, 26, 24, 18, 22, 6, 9, 3, 21, 4, 4, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 5]]


In [8]:
# 디코더의 예측값과 비교하기 위한 실제값 (영어 문장)의 맨 앞에 붙어있는 \t 제거
decoder_target = []
for line in lines.tar:
    t=0
    temp_X = []
    for w in line:
        if t>0:
            temp_X.append(tar_to_index[w])
        t=t+1
    decoder_target.append(temp_X)
print(decoder_target[:5])

[[21, 27, 26, 24, 18, 22, 6, 9, 3, 21, 4, 4, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 5], [21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 23, 8, 21, 16, 11, 10, 25, 6, 9, 19, 18, 17, 21, 7, 21, 12, 19, 27, 2, 26, 1, 9, 21, 5], [21, 16, 14, 9, 2, 11, 17, 21, 23, 8, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 7, 21, 9, 13, 18, 9, 19, 1, 24, 21, 5], [21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 4, 4, 21, 27, 2, 9, 3, 20, 21, 5], [21, 27, 26, 24, 18, 22, 6, 9, 3, 21, 4, 4, 21, 16, 27, 14, 9, 27, 20, 22, 26, 18, 17, 21, 5]]


In [9]:
# 패딩을 위해서 한국어 문장과 영어 문장 각각에 대해서,
# 가장 길이가 긴 샘플의 길이를 알아보겠습니다.
max_src_len = max([len(line) for line in lines.src])
max_tar_len = max([len(line) for line in lines.tar])
print(max_src_len)
print(max_tar_len)

30
37


In [10]:
# 여기서는 가장 긴 샘플의 길이에 맞춰서,
# 한국어 데이터의 샘플은 전부 길이가 25가 되도록 패딩하고,
# 영어 데이터의 샘플은 전부 길이가 76이 되도록 패딩합니다.
from tensorflow.keras.preprocessing.sequence import pad_sequences
encoder_input = pad_sequences(encoder_input, maxlen=max_src_len, padding='post')
decoder_input = pad_sequences(decoder_input, maxlen=max_tar_len, padding='post')
decoder_target = pad_sequences(decoder_target, maxlen=max_tar_len, padding='post')

In [11]:
# 이제 모든 값에 대해서 원-핫 인코딩 수행
# 실제값, 입력값 모두 원-핫 벡터 사용
from tensorflow.keras.utils import to_categorical
encoder_input = to_categorical(encoder_input)
decoder_input = to_categorical(decoder_input)
decoder_target = to_categorical(decoder_target)

# 데이터 전처리 끝!

In [12]:
# seq2seq 기계 번역기 훈련시키기 (교사 강요 사용)

In [13]:
from tensorflow.keras.layers import Input, LSTM, Embedding, Dense
from tensorflow.keras.models import Model

encoder_inputs = Input(shape=(None, src_vocab_size))
encoder_lstm = LSTM(units=256, return_state=True)
encoder_outputs, state_h, state_c = encoder_lstm(encoder_inputs)
# encoder_outputs도 같이 리턴받기는 했지만 여기서는 필요없으므로 이 값은 버림.
encoder_states = [state_h, state_c]
# LSTM은 바닐라 RNN과는 달리 상태가 두 개. 바로 은닉 상태와 셀 상태.

In [14]:
decoder_inputs = Input(shape=(None, tar_vocab_size))
decoder_lstm = LSTM(units=256, return_sequences=True, return_state=True)
decoder_outputs, _, _= decoder_lstm(decoder_inputs, initial_state=encoder_states)
# 디코더의 첫 상태를 인코더의 은닉 상태, 셀 상태로 합니다.
decoder_softmax_layer = Dense(tar_vocab_size, activation='softmax')
decoder_outputs = decoder_softmax_layer(decoder_outputs)

model = Model([encoder_inputs, decoder_inputs], decoder_outputs)
model.compile(optimizer="rmsprop", loss="categorical_crossentropy")

In [15]:
# 50 에포크 학습
model.fit(x=[encoder_input, decoder_input], y=decoder_target, batch_size=64, epochs=50, validation_split=0.2)

Train on 1152 samples, validate on 288 samples
Epoch 1/50
1152/1152 [==============================] - 6s 5ms/sample - loss: 2.7114 - val_loss: 2.2122
Epoch 2/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 2.1426 - val_loss: 1.9201
Epoch 3/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 1.8922 - val_loss: 1.6717
Epoch 4/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 1.6529 - val_loss: 1.5456
Epoch 5/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 1.3799 - val_loss: 1.3672
Epoch 6/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 1.1875 - val_loss: 1.2647
Epoch 7/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 0.9950 - val_loss: 0.7918
Epoch 8/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 0.7726 - val_loss: 0.6889
Epoch 9/50
1152/1152 [==============================] - 3s 2ms/sample - loss: 0.6743 - val_loss: 0.4886
Epoch 10/50
1152/

In [16]:
# 인코더 정의
encoder_model = Model(inputs=encoder_inputs, outputs=encoder_states)

In [17]:
# 디코더 설계
# 이전 시점의 상태들을 저장하는 텐서
decoder_state_input_h = Input(shape=(256,))
decoder_state_input_c = Input(shape=(256,))
decoder_states_inputs = [decoder_state_input_h, decoder_state_input_c]
decoder_outputs, state_h, state_c = decoder_lstm(decoder_inputs, initial_state=decoder_states_inputs)
# 문장의 다음 단어를 예측하기 위해서 초기 상태(initial_state)를 이전 시점의 상태로 사용. 이는 뒤의 함수 decode_sequence()에 구현
decoder_states = [state_h, state_c]
# 훈련 과정에서와 달리 LSTM의 리턴하는 은닉 상태와 셀 상태인 state_h와 state_c를 버리지 않음.
decoder_outputs = decoder_softmax_layer(decoder_outputs)
decoder_model = Model(inputs=[decoder_inputs] + decoder_states_inputs, outputs=[decoder_outputs] + decoder_states)

In [18]:
# 단어로부터 인덱스를 얻는 것이 아니라 인덱스로부터 단어를 얻을 수 있는
# index_to_src와 index_to_tar 생성
index_to_src = dict((i, char) for char, i in src_to_index.items())
index_to_tar = dict((i, char) for char, i in tar_to_index.items())

# print("index_to_src: ")
# for k, v in index_to_src.items():
#     print(k, v)
    
# print("index_to_tar: ")
# for k, v in index_to_tar.items():
#     print(k, v)

In [21]:
def decode_sequence(input_seq):
    # 입력으로부터 인코더의 상태를 얻음
    states_value = encoder_model.predict(input_seq)

    # <SOS>에 해당하는 원-핫 벡터 생성
    target_seq = np.zeros((1, 1, tar_vocab_size))
    target_seq[0, 0, tar_to_index['\t']] = 1.

    stop_condition = False
    decoded_sentence = ""

    # stop_condition이 True가 될 때까지 루프 반복
    while not stop_condition:
        # 이점 시점의 상태 states_value를 현 시점의 초기 상태로 사용
        output_tokens, h, c = decoder_model.predict([target_seq] + states_value)

        # 예측 결과를 문자로 변환
        sampled_token_index = np.argmax(output_tokens[0, -1, :])
        sampled_char = index_to_tar[sampled_token_index]

        # 현재 시점의 예측 문자를 예측 문장에 추가
        decoded_sentence += sampled_char

        # <eos>에 도달하거나 최대 길이를 넘으면 중단.
        if (sampled_char == '\n' or
#             len(decoded_sentence) > max_tar_len):
            len(decoded_sentence) > max_tar_len - 10):
            stop_condition = True

        # 현재 시점의 예측 결과를 다음 시점의 입력으로 사용하기 위해 저장
        target_seq = np.zeros((1, 1, tar_vocab_size))
        target_seq[0, 0, sampled_token_index] = 1.

        # 현재 시점의 상태를 다음 시점의 상태로 사용하기 위해 저장
        states_value = [h, c]

    return decoded_sentence

In [22]:
import numpy as np
for seq_index in [3, 50, 100, 300, 1001]: # 입력 문장의 인덱스
    input_seq = encoder_input[seq_index: seq_index + 1]
    decoded_sentence = decode_sequence(input_seq)
    print(35 * "-")
    print('입력 문장:', lines.src[seq_index])
    print('정답 문장:', lines.tar[seq_index][1:len(lines.tar[seq_index])-1]) # '\t'와 '\n'을 빼고 출력
    print('번역기가 번역한 문장:', decoded_sentence[:len(decoded_sentence)-1]) # '\n'을 빼고 출력


-----------------------------------
입력 문장: Checkout is done by clerk.
정답 문장:  (checkout) -- clerk 
번역기가 번역한 문장:  (checkout) -- clerk 
-----------------------------------
입력 문장: Help extends checkout.
정답 문장:  (help) .> (checkout) : extends 
번역기가 번역한 문장:  (help) .> (checkout) : ext
-----------------------------------
입력 문장: The customer does checkout.
정답 문장:  customer -- (checkout) 
번역기가 번역한 문장:  customer -- (checkout) 
-----------------------------------
입력 문장: The customer does checkout.
정답 문장:  customer -- (checkout) 
번역기가 번역한 문장:  customer -- (checkout) 
-----------------------------------
입력 문장: The checkouts require payment.
정답 문장:  (checkout) .> (payment) : include 
번역기가 번역한 문장:  (checkout) .> (payment) : 
